In [ ]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time


In [ ]:
v, f = igl.read_triangle_mesh('data/hand.off')
labels = np.load('hand_label.npy').astype(int)

In [ ]:
v_targ = v.copy()

pos_f_saver = np.zeros((labels.max() + 1, 6))
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    v_targ[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    ## Computation Within
    t0 = time.time()
    v_deformed = pos_f.deformer(v_targ)
    
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x
segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
translate_widget = {i:iw.FloatSlider(min=-30, max=30, value=0) 
                    for i in 'xyz'}
rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                 for a in 'αβγ'}

def update_seg(*args):
    (translate_widget['x'].value,translate_widget['y'].value,
    translate_widget['z'].value,
    rotate_widget['α'].value,rotate_widget['β'].value,
    rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
segment_widget.observe(update_seg, 'value')

p = mp.plot(v_targ, f, c=labels)
iw.interact(pos_f,
            s = segment_widget,
            **translate_widget,
            **rotate_widget)